<a href="https://colab.research.google.com/github/azario0/tiktaktoe-llm/blob/main/en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports necessary libraries: <br>
google.generativeai for AI interactions and random for random moves.<br>
Configures the AI model using an API key and selects the model 'models/gemini-2.0-flash-exp'.

In [1]:
import google.generativeai as genai
import random

# Configure Gemini AI
genai.configure(api_key='YOUR_API_KEY')
model = genai.GenerativeModel('models/gemini-2.0-flash-exp')


# Defines a function to display the current <br>state of the Tic-Tac-Toe board.

Prints the board in a readable format with separators.

In [2]:
def print_board(board):
    """Displays the Tic-Tac-Toe board."""
    print("\n")
    for i, row in enumerate(board):
        print(" | ".join(row))
        if i < 2:
            print("---------")

# Checks if the specified player has won the game.

Verifies all possible winning combinations: rows, columns, and diagonals.

In [3]:
def is_winner(board, player):
    """Checks if the given player has won the game."""
    # Check rows
    for row in board:
        if all(cell == player for cell in row):
            return True

    # Check columns
    for col in range(3):
        if all(board[row][col] == player for row in range(3)):
            return True

    # Check diagonals
    if all(board[i][i] == player for i in range(3)) or \
       all(board[i][2-i] == player for i in range(3)):
        return True

    return False


# Determines if the game has ended in a draw.
Checks if all cells on the board are filled without a winner.

In [4]:
def is_draw(board):
    """Checks if the game is a draw."""
    return all(cell != " " for row in board for cell in row)

# Handles the player's input for their move.

Ensures the move is valid (within 1-9 and the selected cell is empty).

Updates the board with the player's move.

In [5]:
def get_player_input(board, player):
    """Gets the player's move and updates the board."""
    while True:
        try:
            print("\nBoard positions:")
            print("1 | 2 | 3\n---------\n4 | 5 | 6\n---------\n7 | 8 | 9")
            move = int(input(f"\nPlayer {player}, enter your move (1-9): ")) - 1
            row, col = divmod(move, 3)
            if 0 <= move < 9 and board[row][col] == " ":
                board[row][col] = player
                return
            else:
                print("Invalid move. That position is already taken or out of range.")
        except ValueError:
            print("Please enter a valid number between 1 and 9.")

# Implements the AI's decision-making <br>process for its move.

Uses the Gemini AI model to generate a move based on the current board state.

If the AI's response is invalid, it selects a random available move.

In [9]:
def get_ai_move(board):
    """Gets the AI's move using Gemini AI or falls back to a strategic move."""
    # First, check if AI can win in the next move
    for i in range(9):
        row, col = divmod(i, 3)
        if board[row][col] == " ":
            board[row][col] = "O"
            if is_winner(board, "O"):
                print(f"AI chooses move {i + 1}")
                return
            board[row][col] = " "  # Undo the move

    # Then, block player's winning move
    for i in range(9):
        row, col = divmod(i, 3)
        if board[row][col] == " ":
            board[row][col] = "X"
            if is_winner(board, "X"):
                board[row][col] = "O"
                print(f"AI chooses move {i + 1}")
                return
            board[row][col] = " "  # Undo the move

    # Try to get a response from Gemini AI
    board_str = "\n".join([" | ".join(row) for row in board])
    prompt = f"""
    You are playing Tic-Tac-Toe.
    Choose an available move from 1-9 (respond with just the number).
    Current board:
    {board_str}
    Board positions:
    1 | 2 | 3
    ---------
    4 | 5 | 6
    ---------
    7 | 8 | 9
    """

    try:
        response = model.generate_content(prompt)
        move = int(response.text.strip())
        row, col = divmod(move - 1, 3)
        if 0 <= move-1 < 9 and board[row][col] == " ":
            board[row][col] = "O"
            print(f"AI chooses move {move}")
            return
    except:
        pass

    # Fallback strategy: prefer center, then corners, then sides
    preferred_moves = [4, 0, 2, 6, 8, 1, 3, 5, 7]  # Center, corners, sides
    for move in preferred_moves:
        row, col = divmod(move, 3)
        if board[row][col] == " ":
            board[row][col] = "O"
            print(f"code chooses move {move + 1}")
            return

# Contains the main game loop for playing <br>Tic-Tac-Toe against the AI.

Initializes the game board and alternates turns between the player and the AI.

Checks for a win or draw after each move and declares the result accordingly.

In [10]:
def play_tic_tac_toe():
    """Main game loop for Tic-Tac-Toe against AI."""
    board = [[" " for _ in range(3)] for _ in range(3)]
    current_player = "X"

    print("\nWelcome to Tic-Tac-Toe!")
    print("You are X, the AI is O.")

    while True:
        print_board(board)

        if current_player == "X":
            get_player_input(board, current_player)
        else:
            get_ai_move(board)

        if is_winner(board, current_player):
            print_board(board)
            print(f"\n{'You' if current_player == 'X' else 'AI'} win! Congratulations!")
            break

        if is_draw(board):
            print_board(board)
            print("\nIt's a draw! Well played!")
            break

        current_player = "O" if current_player == "X" else "X"

# Starts the Tic-Tac-Toe game by calling <br>the play_tic_tac_toe() function.

Outputs the game progress and results to the console.

In [11]:
play_tic_tac_toe()


Welcome to Tic-Tac-Toe!
You are X, the AI is O.


  |   |  
---------
  |   |  
---------
  |   |  

Board positions:
1 | 2 | 3
---------
4 | 5 | 6
---------
7 | 8 | 9

Player X, enter your move (1-9): 1


X |   |  
---------
  |   |  
---------
  |   |  
AI chooses move 5


X |   |  
---------
  | O |  
---------
  |   |  

Board positions:
1 | 2 | 3
---------
4 | 5 | 6
---------
7 | 8 | 9

Player X, enter your move (1-9): 7


X |   |  
---------
  | O |  
---------
X |   |  
AI chooses move 4


X |   |  
---------
O | O |  
---------
X |   |  

Board positions:
1 | 2 | 3
---------
4 | 5 | 6
---------
7 | 8 | 9

Player X, enter your move (1-9): 6


X |   |  
---------
O | O | X
---------
X |   |  
AI chooses move 8


X |   |  
---------
O | O | X
---------
X | O |  

Board positions:
1 | 2 | 3
---------
4 | 5 | 6
---------
7 | 8 | 9

Player X, enter your move (1-9): 2


X | X |  
---------
O | O | X
---------
X | O |  
AI chooses move 3


X | X | O
---------
O | O | X
---------
X | O